In [6]:
import json
import pdb
import os

def get_tweet_text_from_json(file_path):
    events = []
    labels = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
            labels.append(value["label"])
        return events,labels
    
def get_tweet_text_from_json_unlabel(file_path):
    events = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
        return events
    
misinfo_train_events,misinfo_train_labels = get_tweet_text_from_json("train.json")
dev_events,dev_labels = get_tweet_text_from_json("dev.json")
test_unlabel = get_tweet_text_from_json_unlabel("test-unlabelled.json")

In [2]:
import nltk
nltk.download("stopwords")

[nltk_data] Error loading stopwords: HTTP Error 500: Internal Server
[nltk_data]     Error


False

In [9]:
import nltk
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


wnl = WordNetLemmatizer()
tt = TweetTokenizer()
stopwords = set(stopwords.words('english'))
punc = punctuation + u'.,;《》？！“”‘’@#￥%…&×（）——+【】{};；●，。&～、|\s:：'
# exclude = set(punc)

def get_wordnet_pos(treebank_tag):

    if (treebank_tag.startswith('J'))|(treebank_tag =='ADJ'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif (treebank_tag.startswith('R'))|(treebank_tag=='ADV'):
        return wordnet.ADV
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    else:
        return wordnet.NOUN

def preprocess_drop_punc_evnets(events):
#     preprocess_drop_punc_evnets = events
    preprocess_drop_punc_evnets = []
#     pdb.set_trace()
    for event in events:
        event_re = re.sub(r"[{}]+".format(punc)," ",event)
        preprocess_drop_punc_evnets.append(event_re)
    return preprocess_drop_punc_evnets

def preprocess_stopwords_lemma_evnets(events):
    preprocess_BOW_events = []
    preprocess_events = []
    preprocess_events_tokens = []
#     pdb.set_trace()
    after_drop_punc_evnets = preprocess_drop_punc_evnets(events)
    for event in after_drop_punc_evnets:
        dic_bow = defaultdict(int)
        tokenize_words = tt.tokenize(event)
#         rep = [abbr_dict[x] if x in abbr_dict else x for x in tokenize_words]
        words_pos = nltk.pos_tag(tokenize_words, tagset="universal")
#         pdb.set_trace()
        filter_event = [(w,pos) for (w,pos) in words_pos if w.lower() not in stopwords]
        lemma_event = []
        for (word,pos) in filter_event:
#             pdb.set_trace()
            word_lemma = wnl.lemmatize(word.lower(),get_wordnet_pos(pos))
            lemma_event.append(word_lemma)
            dic_bow[word_lemma] += 1
        preprocess_events_tokens.append(lemma_event,)
        preprocess_events.append(' '.join(lemma_event))
        preprocess_BOW_events.append(dic_bow)
    return preprocess_events_tokens,preprocess_events,preprocess_BOW_events

preprocess_train_events_tokens,preprocessed_train_events,preprocessed_train_BOW_events = preprocess_stopwords_lemma_evnets(misinfo_train_events)
preprocess_dev_events_tokens,preprocessed_dev_events,preprocessed_dev_BOW_events = preprocess_stopwords_lemma_evnets(dev_events)
preprocess_test_events_tokens,preprocessed_test,preprocessed_test_BOW = preprocess_stopwords_lemma_evnets(test_unlabel)

In [80]:
import gensim
from gensim import corpora
all_text_tokens = preprocess_train_events_tokens + preprocess_train_events_tokens + preprocess_test_events_tokens
dictionary = corpora.Dictionary(all_text_tokens)
dictionary.filter_extremes(no_below = 10, no_above = 0.35)
dictionary.compactify()
doc_term_matrix = [dictionary.doc2bow(event) for event in all_text_tokens]

In [81]:
Lda = gensim.models.ldamodel.LdaModel

ldamodel = Lda(doc_term_matrix, num_topics=50, id2word = dictionary, passes=50)

In [84]:
print(ldamodel.print_topics(num_topics = 1, num_words = 20))

[(7, '0.025*"meat" + 0.022*"eat" + 0.019*"zealand" + 0.016*"carter" + 0.016*"farmer" + 0.015*"r" + 0.012*"house" + 0.011*"beef" + 0.011*"food" + 0.010*"perry" + 0.010*"amendment" + 0.010*"bob" + 0.009*"committee" + 0.009*"pentagon" + 0.008*"diet" + 0.008*"burger" + 0.008*"fuel" + 0.008*"agenda" + 0.007*"livestock" + 0.006*"alternative"')]


In [45]:
top_topics = ldamodel.get_document_topics(doc_term_matrix[1171],minimum_probability = 0.0)

In [85]:
train_vecs = []
for i in range(len(preprocess_train_events_tokens)):
    top_topics = ldamodel.get_document_topics(doc_term_matrix[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(3)]
#     topic_vec.extend([preprocess_train_events_tokens[i].real_counts]) # counts of reviews for restaurant
#     topic_vec.extend([len(preprocess_train_events_tokens[i])]) # length review
    train_vecs.append(topic_vec)

In [86]:
dev_vecs = []
for i in range(len(preprocess_dev_events_tokens)):
    top_topics = ldamodel.get_document_topics(doc_term_matrix[i+len(preprocess_train_events_tokens)], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(3)]
#     topic_vec.extend([preprocess_train_events_tokens[i].real_counts]) # counts of reviews for restaurant
#     topic_vec.extend([len(preprocess_train_events_tokens[i])]) # length review
    dev_vecs.append(topic_vec)
    
test_vecs = []
for i in range(len(preprocess_test_events_tokens)):
    top_topics = ldamodel.get_document_topics(doc_term_matrix[i+len(preprocess_train_events_tokens)+len(preprocess_dev_events_tokens)], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(3)]
#     topic_vec.extend([preprocess_train_events_tokens[i].real_counts]) # counts of reviews for restaurant
#     topic_vec.extend([len(preprocess_train_events_tokens[i])]) # length review
    test_vecs.append(topic_vec)



In [87]:
print(len(test_vecs))

1410


In [88]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
def evaluate_result(groundtruth, prediction):
    prediction = prediction.tolist()
    for i,item in enumerate(prediction):
        if item == -1:
            prediction[i] = 0
    p, r, f, _ = precision_recall_fscore_support(groundtruth, prediction, pos_label=1, average="binary")
#     pdb.set_trace()
    print("Performance on the positive class (documents with misinformation):")
    print("Precision =", p)
    print("Recall    =", r)
    print("F1        =", f)
    print(accuracy_score(dev_labels,prediction))
    
def transfer_output(preds):
    test_dic = {}
    for i,test_pre in enumerate(preds):
        if test_pre == -1:
            dic = {}
            dic['label'] = 0
    #         pdb.set_trace()
            test_dic['test-%s'%(i)] = dic
        else:
            dic = {}
    #         pdb.set_trace()
            dic['label'] = 1
            test_dic['test-%s'%(i)] = dic
    jstr = json.dumps(test_dic,ensure_ascii = False)
    # print(jstr)
    with open('test-output.json','w') as f:
        f.write(jstr)

In [89]:
from sklearn import svm

nus = [0.001, 0.01, 0.1, 1]
gammas = [0.001, 0.01, 0.1, 1]
kernels = ['rbf','linear']
for kernel in kernels:
    for nu in nus:
        for gamma in gammas:
            ocs = svm.OneClassSVM(gamma=gamma,kernel = kernel, nu = nu).fit(train_vecs)
            y_pre = ocs.predict(dev_vecs)
            print(ocs)
            evaluate_result(dev_labels,y_pre)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5
Recall    = 1.0
F1        = 0.6666666666666666
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5
Recall    = 1.0
F1        = 0.6666666666666666
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5
Recall    = 1.0
F1        = 0.6666666666666666
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='rbf',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMe

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=1, kernel='rbf',
            max_iter=-1, nu=1, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.0
Recall    = 0.0
F1        = 0.0
0.5
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.001, kernel='linear',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5882352941176471
Recall    = 0.2
F1        = 0.29850746268656714
0.53
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.01, kernel='linear',
            max_iter=-1, nu=0.001, shrinking=True, tol=0.001, verbose=False)
Performance on the positive class (documents with misinformation):
Precision = 0.5882352941176471
Recall    = 0.2
F1        = 0.29850746268656714
0.53
OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='linear',
            max_iter=-1, nu=0.001, shrinking=T

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
ocs = svm.OneClassSVM(gamma=0.001,kernel = 'linear', nu = 0.01).fit(train_vecs)
test_pres = ocs.predict(test_vecs)

print(test_pres)

[1 1 1 ... 1 1 1]


In [92]:
from sklearn.neighbors import LocalOutlierFactor
n_neighborss = [10,20,30,40,100]
contaminations = [0.1,0.3,0.5]
for n_neighbor in n_neighborss:
    for contamination in contaminations:
        clf = LocalOutlierFactor(n_neighbors = n_neighbor, novelty=True, contamination=contamination)
        clf.fit(train_vecs)
        y_pre = clf.predict(dev_vecs)
        print(clf)
        evaluate_result(dev_labels,y_pre)

LocalOutlierFactor(algorithm='auto', contamination=0.1, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.4945054945054945
Recall    = 0.9
F1        = 0.6382978723404256
0.49
LocalOutlierFactor(algorithm='auto', contamination=0.3, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.4675324675324675
Recall    = 0.72
F1        = 0.5669291338582677
0.45
LocalOutlierFactor(algorithm='auto', contamination=0.5, leaf_size=30,
                   metric='minkowski', metric_params=None, n_jobs=None,
                   n_neighbors=10, novelty=True, p=2)
Performance on the positive class (documents with misinformation):
Precision = 0.4444444444444444
Recall    = 0

In [93]:
clf = LocalOutlierFactor(n_neighbors = 40, novelty=True, contamination=0.1)
clf.fit(train_vecs)
test_pres = clf.predict(test_vecs)
print(test_pres)

[ 1 -1  1 ...  1  1  1]


In [94]:
print(sum(test_pres))

1046


In [70]:
transfer_output(test_pres)

In [96]:
# 0.556
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
tfidfvector = TfidfVectorizer()
train_tfidfvector = tfidfvector.fit_transform(preprocessed_train_events)
dev_tfidfvector = tfidfvector.transform(preprocessed_dev_events)
test_tfidfvector = tfidfvector.transform(preprocessed_test)
print(len(tfidfvector.vocabulary_))

from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors = 10, novelty=True, contamination=0.3)
clf.fit(train_tfidfvector)
test_pres = clf.predict(test_tfidfvector)
print(test_pres)
# transfer_output(test_pres)

19561
[-1 -1 -1 ... -1  1 -1]


In [97]:
print(sum(test_pres))

-948
